# sumaryzacja tekstu

In [9]:
from google.colab import drive


drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [10]:
!nvidia-smi

Sun Nov 16 13:13:15 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   32C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [11]:
# 1. Force uninstall both to clear the conflict
!pip uninstall -y unsloth unsloth_zoo

# 2. Install Unsloth Zoo DIRECTLY from Git (to get 'tiled_mlp')
!pip install --upgrade --no-cache-dir "unsloth_zoo @ git+https://github.com/unslothai/unsloth-zoo.git"

# 3. Install Unsloth main library from Git
!pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

Found existing installation: unsloth 2025.11.3
Uninstalling unsloth-2025.11.3:
  Successfully uninstalled unsloth-2025.11.3
Found existing installation: unsloth_zoo 2025.11.4
Uninstalling unsloth_zoo-2025.11.4:
  Successfully uninstalled unsloth_zoo-2025.11.4
  Cloning https://github.com/unslothai/unsloth-zoo.git to /tmp/pip-install-k4hb8o1k/unsloth-zoo_d4e2d38b231247598e0c71408ed25fba
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth-zoo.git /tmp/pip-install-k4hb8o1k/unsloth-zoo_d4e2d38b231247598e0c71408ed25fba
  Resolved https://github.com/unslothai/unsloth-zoo.git to commit b60fd19e4e1c27014553f7194669d66cdb96db2c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 34.4 MB/s eta 0:00:00
  Created wheel for unsloth_zoo: filename=unsloth_zoo-2025.11.4-py3-none-any.whl size=281896 sha256=cb3e29bbcc99abf3b

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-a71jz1ob/unsloth_1d9ca806c370492199a00e3d5f67260e
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-a71jz1ob/unsloth_1d9ca806c370492199a00e3d5f67260e
  Resolved https://github.com/unslothai/unsloth.git to commit 38bdbed0bf65230bf076fd7e59583006348bde47
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2025.11.3-py3-none-any.whl size=357217 sha256=24a40b0b720deb02dde55c9b68e02a7c7bc75f1d4f65ca8bfb5019557112b043
  Stored in directory: /tmp/pip-ephem-wheel-cache-jboxqvr8/wheels/60/3e/1f/e576c07051d90cf64b6a41434d87ccf4db33fafd5343bf5de0
Successfully built unsloth


In [12]:
!pip install "datasets<3.0.0"


  Using cached datasets-2.21.0-py3-none-any.whl.metadata (21 kB)
Using cached datasets-2.21.0-py3-none-any.whl (527 kB)
  Attempting uninstall: datasets
    Found existing installation: datasets 4.3.0
    Uninstalling datasets-4.3.0:
      Successfully uninstalled datasets-4.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
trl 0.24.0 requires datasets>=3.0.0, but you have datasets 2.21.0 which is incompatible.
unsloth-zoo 2025.11.4 requires datasets!=4.0.*,!=4.1.0,<4.4.0,>=3.4.1, but you have datasets 2.21.0 which is incompatible.


In [13]:
!pip install bert-extractive-summarizer

In [14]:
!pip install pytorch-ignite
!pip install torchmetrics

## importowanie datesetu

In [15]:
from datasets import load_dataset


ds = load_dataset("xsum", trust_remote_code=True)
ds= ds.filter(lambda example: len(example["document"]) < 750)
split_result = ds["test"].train_test_split(test_size=2, seed=42)
shots = split_result["test"]
ds["test"] = split_result["train"]

Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

Filter:   0%|          | 0/204045 [00:00<?, ? examples/s]

Filter:   0%|          | 0/11332 [00:00<?, ? examples/s]

Filter:   0%|          | 0/11334 [00:00<?, ? examples/s]

In [16]:
print(ds)
print(shots)

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 32053
    })
    validation: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 1746
    })
    test: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 1727
    })
})
Dataset({
    features: ['document', 'summary', 'id'],
    num_rows: 2
})


In [17]:
from summarizer import Summarizer
model = Summarizer()

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [18]:
from ignite.metrics import Rouge

m = Rouge(variants=["L", 2], multiref="best")

candidate = "the cat is not there".split()
references = [
    "the cat is on the mat".split(),
    "there is a cat on the mat".split()
]

m.update(([candidate], [references]))

print(m.compute())

{'Rouge-L-P': 0.6, 'Rouge-L-R': 0.5, 'Rouge-L-F': 0.5, 'Rouge-2-P': 0.5, 'Rouge-2-R': 0.4, 'Rouge-2-F': 0.4}


In [19]:
from pprint import pprint
from torchmetrics.text.bert import BERTScore
preds = """
The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.
Repair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.
Trains on the west coast mainline face disruption due to damage at the Lamington Viaduct.
Many businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.
First Minister Nicola Sturgeon visited the area to inspect the damage.
The waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare.
Jeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit.
However, she said more preventative work could have been carried out to ensure the retaining wall did not fail.
"It is difficult but I do think there is so much publicity for Dumfries and the Nith - and I totally appreciate that - but it is almost like we're neglected or forgotten," she said.
"That may not be true but it is perhaps my perspective over the last few days.
"Why were you not ready to help us a bit more when the warning and the alarm alerts had gone out?"
Meanwhile, a flood alert remains in place across the Borders because of the constant rain.
Peebles was badly hit by problems, sparking calls to introduce more defences in the area.
Scottish Borders Council has put a list on its website of the roads worst affected and drivers have been urged not to ignore closure signs.
The Labour Party's deputy Scottish leader Alex Rowley was in Hawick on Monday to see the situation first hand.
He said it was important to get the flood protection plan right but backed calls to speed up the process.
"I was quite taken aback by the amount of damage that has been done," he said.
"Obviously it is heart-breaking for people who have been forced out of their homes and the impact on businesses."
He said it was important that "immediate steps" were taken to protect the areas most vulnerable and a clear timetable put in place for flood prevention plans.
Have you been affected by flooding in Dumfries and Galloway or the Borders? Tell us about your experience of the situation and how it was handled. Email us on selkirk.news@bbc.co.uk or dumfries@bbc.co.uk.
"""
result = model(preds)
# print(result)
target ="""
Clean-up operations are continuing across the Scottish Borders and Dumfries and Galloway after flooding caused by Storm Frank.
"""
print(f'target: {target}')
print(f'result: {result}')
bertscore = BERTScore(model_name_or_path="microsoft/deberta-v3-large", lang='en')
pprint(bertscore([result], [target]))

target: 
Clean-up operations are continuing across the Scottish Borders and Dumfries and Galloway after flooding caused by Storm Frank.

result: The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed. Trains on the west coast mainline face disruption due to damage at the Lamington Viaduct. First Minister Nicola Sturgeon visited the area to inspect the damage. "That may not be true but it is perhaps my perspective over the last few days. He said it was important to get the flood protection plan right but backed calls to speed up the process.


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/580 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/874M [00:00<?, ?B/s]

The following layers were not sharded: embeddings.LayerNorm.bias, encoder.layer.*.attention.self.query_proj.weight, embeddings.word_embeddings.weight, encoder.layer.*.attention.self.value_proj.bias, encoder.layer.*.attention.output.dense.bias, encoder.layer.*.output.dense.bias, encoder.layer.*.attention.self.key_proj.weight, encoder.layer.*.output.LayerNorm.weight, encoder.layer.*.attention.self.query_proj.bias, encoder.layer.*.attention.output.LayerNorm.weight, encoder.layer.*.intermediate.dense.bias, embeddings.LayerNorm.weight, encoder.layer.*.attention.self.value_proj.weight, encoder.layer.*.output.LayerNorm.bias, encoder.layer.*.intermediate.dense.weight, encoder.layer.*.attention.self.key_proj.bias, encoder.rel_embeddings.weight, encoder.layer.*.output.dense.weight, encoder.LayerNorm.weight, encoder.layer.*.attention.output.LayerNorm.bias, encoder.LayerNorm.bias, encoder.layer.*.attention.output.dense.weight


model.safetensors:   0%|          | 0.00/874M [00:00<?, ?B/s]

{'f1': tensor(0.7959), 'precision': tensor(0.7827), 'recall': tensor(0.8095)}


In [20]:
num_workers = 1
batch_size = 1

In [21]:
from torch.utils.data import DataLoader
import torch
test_dataset = ds["test"]
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
print(test_dataset)


Dataset({
    features: ['document', 'summary', 'id'],
    num_rows: 1727
})


In [22]:
def bert_score_loss(batch, model, bert: bool, tokenizer: any, prompt = None, promptsuff=None):
    if bert:
        predictions = [model(doc, ratio=0.1) for doc in batch['document']]
    else:
        docs = [doc for doc in batch['document']]
        prompted  = [f'{prompt}{doc}{promptsuff}'for doc in batch['document']]
        inputs = tokenizer(
        prompted,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=4096
    )
        device = next(model.parameters()).device
        inputs = {k: v.to(device) for k, v in inputs.items()}
        summary_ids = model.generate(
        inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_new_tokens=50,
        do_sample=True,
        top_k=0,
        temperature=0.5,
    )
        input_length = inputs["input_ids"].shape[1]
        new_tokens = summary_ids[:, input_length:]
        predictions = tokenizer.batch_decode(
            new_tokens, skip_special_tokens=True
        )
    print(f"Doc: {batch['document'][0][:100]}")
    print(f"Prediction: {predictions[0]}")
    print(f"Target: {batch['summary'][0]}")
    predictions = [pred if pred.strip() else "." for pred in predictions]
    for i, pred in enumerate(predictions):
        if pred.strip() == "":
            print("EMPTY PREDICTION")
            print(batch['document'][i])
    return bertscore(predictions, batch["summary"])

In [23]:
def batch_to_device(batch: dict, device) -> dict:
    return {k: v.to(device) for k, v in batch.items()}

In [24]:
import torch
import torch.nn as nn
from torch import Tensor
def calc_loss_batch(batch: dict, model: nn.Module) -> Tensor:
    logits = model(batch['input_ids'])
    loss = torch.nn.functional.cross_entropy(logits.flatten(0, 1), batch['labels'].flatten())
    return loss

In [25]:
def calc_loss_loader(data_loader, model, device, num_batches: int, bert: bool, tokenizer: any,prompt = None, promptsuff = None) -> float:
    total_loss = {'precision': [], 'recall': [], 'f1': []}
    for i, batch in enumerate(data_loader):
        if i >= num_batches:
            break
        # batch = batch_to_device(batch, device)
        loss = bert_score_loss(batch, model, bert, tokenizer, prompt, promptsuff)
        total_loss['precision'].append(loss['precision'].unsqueeze(0))
        total_loss['recall'].append(loss['recall'].unsqueeze(0))
        total_loss['f1'].append(loss['f1'].unsqueeze(0))
    total_loss['precision'] = torch.cat(total_loss['precision'])
    total_loss['recall'] = torch.cat(total_loss['recall'])
    total_loss['f1'] = torch.cat(total_loss['f1'])
    print(total_loss)
    return (sum(total_loss['precision'])/ len(total_loss['precision']),sum(total_loss['recall'])/ len(total_loss['recall']),sum(total_loss['f1'])/ len(total_loss['f1']))

In [26]:
def evaluate_model(model: nn.Module, loader: DataLoader,
                    device, eval_iter: int, bert: bool, tokenizer: any, prompt="", promptsuff ="") -> tuple[float, float]:
    if not bert:
      model.eval()
    with torch.no_grad():
        loss = calc_loss_loader(loader, model, device, num_batches=eval_iter, bert=bert, tokenizer=tokenizer, prompt=prompt, promptsuff=promptsuff)
    return loss

In [27]:
from torch import nn
loss = evaluate_model(model, test_loader, device='cuda', eval_iter=1, bert=True, tokenizer=None)
print(loss)



Doc: Tommy Sheldon, from Southampton, died two weeks after being pulled from the inferno by a passer-by i
Prediction: Tommy Sheldon, from Southampton, died two weeks after being pulled from the inferno by a passer-by in Merdon Castle Lane, Hursley, Hampshire, on 11 August. She was remanded in custody when she appeared at Winchester Crown Court.
Target: The mother of a five-year-old boy has been charged with his murder after he died in a car fire.


/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
The following layers were not sharded: embeddings.LayerNorm.bias, encoder.layer.*.attention.self.query_proj.weight, embeddings.word_embeddings.weight, encoder.layer.*.attention.self.value_proj.bias, encoder.layer.*.attention.output.dense.bias, encoder.layer.*.output.dense.bias, encoder.layer.*.attention.self.key_proj.weight, encoder.layer.*.output.LayerNorm.weight, encoder.layer.*.attention.self.query_proj.bias, encoder.layer.*.attention.output.LayerNorm.weight, encoder.layer.*.intermediate.dense.

{'precision': tensor([0.7766]), 'recall': tensor([0.8248]), 'f1': tensor([0.8000])}
(tensor(0.7766), tensor(0.8248), tensor(0.8000))


In [28]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
tokenizer.padding_side = 'left'
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    # Some models also require explicitly setting the pad_token_id
    tokenizer.pad_token_id = tokenizer.eos_token_id
model2 = AutoModelForCausalLM.from_pretrained("openai-community/gpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

The following layers were not sharded: transformer.h.*.mlp.c_proj.weight, transformer.h.*.attn.c_attn.weight, transformer.h.*.attn.c_attn.bias, transformer.h.*.mlp.c_proj.bias, transformer.wpe.weight, lm_head.weight, transformer.ln_f.weight, transformer.h.*.attn.c_proj.bias, transformer.wte.weight, transformer.h.*.mlp.c_fc.weight, transformer.h.*.mlp.c_fc.bias, transformer.ln_f.bias, transformer.h.*.ln_*.bias, transformer.h.*.attn.c_proj.weight, transformer.h.*.ln_*.weight


generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [30]:
def generate_prompt(tokenizer, size):
    prompt = ""
    shots_loader = DataLoader(shots, batch_size=100000, shuffle=True, num_workers=num_workers)
    for batch in shots_loader:
        for i in range(len(batch['document'])):
            prompt += f'{i+1}. {batch['document'][i]}\n###Summary\n{batch['summary'][i]}'
            prompt += tokenizer.eos_token
    return prompt

prompt = generate_prompt(tokenizer, 3)
print(prompt)

UnboundLocalError: cannot access local variable 'prompt' where it is not associated with a value

In [ ]:
summary = "\n###Summary\n"

In [ ]:
print(prompt)
print(evaluate_model(model2, test_loader, device='cuda', eval_iter=1, bert=False, tokenizer=tokenizer, prompt = prompt, promptsuff=summary))


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer3 = AutoTokenizer.from_pretrained("Qwen/Qwen1.5-0.5B")
tokenizer3.padding_side = 'left'
if tokenizer3.pad_token is None:
    tokenizer3.pad_token = tokenizer3.eos_token
    # Some models also require explicitly setting the pad_token_id
    tokenizer3.pad_token_id = tokenizer3.eos_token_id
model3 = AutoModelForCausalLM.from_pretrained("Qwen/Qwen1.5-0.5B")




In [ ]:
prompt3 = generate_prompt(tokenizer3, 0)
print(evaluate_model(model3, test_loader, device='cuda', eval_iter=1, bert=False, tokenizer=tokenizer3, prompt=prompt3, promptsuff = summary))

In [ ]:
prompt4 = "Summarize this text. Don't hallucinate. Be thorough and explicit. Always provide an anwer What is this text about? "
print(
    evaluate_model(
        model3,
        test_loader,
        device="cuda",
        eval_iter=1,
        bert=False,
        tokenizer=tokenizer3,
        prompt=prompt4,
    )
)

In [ ]:
print(
    evaluate_model(
        model2,
        test_loader,
        device="cuda",
        eval_iter=2,
        bert=False,
        tokenizer=tokenizer,
        prompt=prompt4,
    )
)

In [ ]:
import torch

print(torch.cuda.is_available())

In [ ]:
# !pip install -U bitsandbytes

In [ ]:
from unsloth import FastLanguageModel

ft_model, ft_tokenzier = FastLanguageModel.from_pretrained(
    model_name="Qwen/Qwen1.5-0.5B", max_seq_length=4096, load_in_4bit=False
)

In [ ]:
ft_ds = ds


def merge_columns(example):
    example["text"] = (
        f"{example['document']}\n### Summary\n{example['summary']}"
    )
    return example

ft_ds = ft_ds.map(merge_columns)

In [ ]:
from trl import SFTTrainer, SFTConfig
ft_model = FastLanguageModel.get_peft_model(
    ft_model,
    r=32,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                  "gate_proj", "up_proj", "down_proj",],
    lora_alpha=16,
    lora_dropout=0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state=1234,
    use_rslora = False,
    loftq_config = None,

)
trainer = SFTTrainer(
    model=ft_model,
    train_dataset=ft_ds["train"],
    eval_dataset=ft_ds["validation"],
    tokenizer=ft_tokenzier,
    args=SFTConfig(
        max_seq_length=4096,
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=10,
        epochs=1,
        logging_steps=1,
        output_dir="outputs",
        optim="adamw_8bit",
        seed=3407,
        eval_strategy="steps",
        eval_steps=200,
    ),
)
trainer.train()

In [ ]:
evaluate_model(ft_model, test_loader, 'cuda', 50, bert=False, tokenizer = ft_tokenzier, promptsuff = "\n### Summary\n")

In [ ]:
ft_model.save('ft')